In [1]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os
from typing import Optional
from dotenv import load_dotenv
from IPython.display import  Image, display

In [2]:
load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY')

In [3]:
# Modelo LLM (pode usar gpt-4o, gpt-3.5, etc)
llm = ChatOpenAI(model="gpt-4.1-mini-2025-04-14", openai_api_key=openai_api_key)

class State(dict):
    mensagem: str
    sentimento: str
    resultado: str
# Estado compartilhado entre os nós
def criar_estado_inicial(mensagem_usuario):
    return {"mensagem": mensagem_usuario, "sentimento": None, "resultado": None}

# Nó 1: Analisar o sentimento
def analisar_sentimento(state):
    prompt = ChatPromptTemplate.from_template(
        "Classifique o sentimento da seguinte mensagem como 'positivo' ou 'negativo': {mensagem}. Use apenas uma palavra."
    )
    resposta = llm.invoke(prompt.format_messages(mensagem=state["mensagem"]))
    state["sentimento"] = resposta.content.lower()
    return state

# Nó 2: Tratar mensagem positiva
def tratar_positivo(state):
    state["resultado"] = "Resposta positiva detectada. Agradecendo ao usuário."
    return state

# Nó 3: Tratar mensagem negativa
def tratar_negativo(state):
    state["resultado"] = "Resposta negativa detectada. Escalando para atendimento humano."
    return state

# Lógica condicional: para onde ir após a análise
def escolher_proximo(state):
    if "positivo" in state["sentimento"]:
        return "tratar_positivo"
    else:
        return "tratar_negativo"


C:\Users\alber\AppData\Local\Temp\ipykernel_22776\215423877.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini-2025-04-14", openai_api_key=openai_api_key)


In [4]:
# Construção do grafo
graph = StateGraph(State)
graph.add_node("analisar_sentimento", analisar_sentimento)
graph.add_node("tratar_positivo", tratar_positivo)
graph.add_node("tratar_negativo", tratar_negativo)
graph.set_entry_point("analisar_sentimento")
graph.add_conditional_edges("analisar_sentimento", escolher_proximo)
graph.add_edge("tratar_positivo", END)
graph.add_edge("tratar_negativo", END)

# Compilar e executar
app = graph.compile()

In [5]:
# Exemplo de uso:
estado_inicial = criar_estado_inicial("Estou muito satisfeito com o atendimento!")
app.invoke(estado_inicial)

{'mensagem': 'Estou muito satisfeito com o atendimento!',
 'sentimento': 'positivo',
 'resultado': 'Resposta positiva detectada. Agradecendo ao usuário.'}

In [6]:
# Exemplo de uso:
estado_inicial = criar_estado_inicial("Que ruim")
app.invoke(estado_inicial)

{'mensagem': 'Que ruim',
 'sentimento': 'negativo',
 'resultado': 'Resposta negativa detectada. Escalando para atendimento humano.'}

In [9]:
app.get_graph().print_ascii()

     +-----------+       
     | __start__ |       
     +-----------+       
            *            
            *            
            *            
+---------------------+  
| analisar_sentimento |  
+---------------------+  
            *            
            *            
            *            
      +---------+        
      | __end__ |        
      +---------+        


In [7]:
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`